In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
# using pytorch native AMP
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
# using Tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="/data/runs")

In [4]:
!pip install wandb
# wandb login

In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: jyotik (use `wandb login --relogin` to force relogin)


True

In [6]:
wandb.init(project="hw09-w251-1GPU")

In [7]:
#import torch.distributed as dist

In [8]:
# Assume that this notebook only sees one GPU.
GPU=0

In [9]:
SEED=1

In [10]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [11]:
torch.cuda.device_count()

1

In [12]:
START_EPOCH = 0

In [13]:
# ARCH = 'resnet18'
ARCH = 'resnet18'
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=256
VAL_BATCH=256
WORKERS=8
TRAINDIR="/data/train"
VALDIR="/data/val"

In [14]:
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [15]:
TRAINDIR="/data/train"
VALDIR="/data/val"

In [16]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [17]:
# how many processes in cluster?
WORLD_SIZE = 2
BACKEND = 'nccl'
# where is the master?
URL = 'tcp://13.57.216.199:1234'

In [18]:
#what is my rank?
#RANK = 1

In [19]:
#dist.init_process_group(backend = BACKEND, init_method= URL,
    #                            world_size= WORLD_SIZE, rank=RANK)

In [20]:
#torch.cuda.set_device('cpu')
torch.cuda.set_device(GPU)

In [21]:
cudnn.benchmark = True

In [22]:
global_step = 0

In [23]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [24]:
def validate(val_loader, model, criterion):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    return top1.avg

In [25]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [26]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [27]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [28]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [29]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [30]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
#cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
#cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
#cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
#cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [31]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [32]:
IMG_SIZE = 224

In [33]:
NUM_CLASSES = 1000

In [34]:
model = models.__dict__[ARCH]()

In [35]:
inf = model.fc.in_features

In [36]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [37]:
model.cuda(GPU)
#model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])
# model = torch.nn.parallel.DistributedDataParallel(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [38]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [39]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [40]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [41]:
#transform_train = transforms.Compose([
#    transforms.Resize(256),
#    transforms.RandomResizedCrop(224),
#    transforms.CenterCrop(224),
#    transforms.RandomHorizontalFlip(),
#    transforms.ToTensor(),
#    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
#])

In [42]:
train_dataset = datasets.ImageFolder(
    TRAINDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
)

In [43]:
#train_dataset = datasets.ImageFolder(
#    TRAINDIR, transform=transform_train)

In [44]:
#transform_val = transforms.Compose([
#    transforms.Resize(256),
#    transforms.CenterCrop(224),
#    transforms.ToTensor(),
#    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
#])

In [45]:
#val_dataset = datasets.ImageFolder(
#    VALDIR, transform=transform_val)

In [46]:
val_dataset = datasets.ImageFolder(
    VALDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])
)

In [47]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [48]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [49]:
best_acc1 = 0

In [50]:
%%time
for epoch in range(START_EPOCH, 2):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    wandb.log({'lr': scheduler.get_last_lr()[0]})

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][   0/5005]	Time 11.059 (11.059)	Data  3.194 ( 3.194)	Loss 6.9886e+00 (6.9886e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.78 (  0.78)
Epoch: [0][  50/5005]	Time  0.317 ( 0.534)	Data  0.002 ( 0.065)	Loss 6.9305e+00 (7.0310e+00)	Acc@1   0.00 (  0.21)	Acc@5   1.17 (  0.79)
Epoch: [0][ 100/5005]	Time  0.319 ( 0.432)	Data  0.002 ( 0.034)	Loss 6.7997e+00 (6.9383e+00)	Acc@1   0.00 (  0.27)	Acc@5   2.34 (  1.13)
Epoch: [0][ 150/5005]	Time  0.320 ( 0.397)	Data  0.002 ( 0.024)	Loss 6.7080e+00 (6.8687e+00)	Acc@1   0.00 (  0.40)	Acc@5   1.56 (  1.49)
Epoch: [0][ 200/5005]	Time  0.344 ( 0.380)	Data  0.003 ( 0.018)	Loss 6.6337e+00 (6.8090e+00)	Acc@1   0.00 (  0.47)	Acc@5   3.12 (  1.78)
Epoch: [0][ 250/5005]	Time  0.320 ( 0.369)	Data  0.002 ( 0.015)	Loss 6.4551e+00 (6.7550e+00)	Acc@1   1.56 (  0.55)	Acc@5   3.91 (  2.11)
Epoch: [0][ 300/5005]	Time  0.319 ( 0.362)	Data  0.002 ( 0.013)	Loss 6.4018e+00 (6.7062e+00)	Acc@1   1.17 (  0.62)	Acc@5   5.08 (  2.40)
Epoch: [0][ 350/5005]	Time  0.345 ( 0.357

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [0][ 950/5005]	Time  0.320 ( 0.339)	Data  0.002 ( 0.006)	Loss 5.6386e+00 (6.2320e+00)	Acc@1   5.47 (  1.83)	Acc@5  15.23 (  6.46)
Epoch: [0][1000/5005]	Time  0.321 ( 0.339)	Data  0.003 ( 0.006)	Loss 5.5779e+00 (6.2026e+00)	Acc@1   3.52 (  1.94)	Acc@5  15.23 (  6.80)
Epoch: [0][1050/5005]	Time  0.348 ( 0.338)	Data  0.003 ( 0.005)	Loss 5.6560e+00 (6.1748e+00)	Acc@1   1.56 (  2.05)	Acc@5  10.55 (  7.11)
Epoch: [0][1100/5005]	Time  0.320 ( 0.338)	Data  0.002 ( 0.005)	Loss 5.5794e+00 (6.1486e+00)	Acc@1   5.47 (  2.15)	Acc@5  16.02 (  7.41)
Epoch: [0][1150/5005]	Time  0.321 ( 0.337)	Data  0.002 ( 0.005)	Loss 5.5507e+00 (6.1212e+00)	Acc@1   4.69 (  2.26)	Acc@5  14.84 (  7.73)
Epoch: [0][1200/5005]	Time  0.321 ( 0.337)	Data  0.002 ( 0.005)	Loss 5.6266e+00 (6.0960e+00)	Acc@1   5.86 (  2.38)	Acc@5  14.06 (  8.04)
Epoch: [0][1250/5005]	Time  0.322 ( 0.337)	Data  0.002 ( 0.005)	Loss 5.5400e+00 (6.0725e+00)	Acc@1   5.86 (  2.48)	Acc@5  18.36 (  8.33)
Epoch: [0][1300/5005]	Time  0.346 ( 0.337

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [1][3100/5005]	Time  0.320 ( 0.333)	Data  0.002 ( 0.003)	Loss 3.4062e+00 (3.6293e+00)	Acc@1  28.91 ( 26.11)	Acc@5  52.73 ( 49.85)
Epoch: [1][3150/5005]	Time  0.346 ( 0.333)	Data  0.002 ( 0.003)	Loss 3.4360e+00 (3.6257e+00)	Acc@1  29.30 ( 26.17)	Acc@5  57.03 ( 49.92)
Epoch: [1][3200/5005]	Time  0.322 ( 0.333)	Data  0.002 ( 0.003)	Loss 3.2509e+00 (3.6221e+00)	Acc@1  33.98 ( 26.22)	Acc@5  58.98 ( 50.00)
Epoch: [1][3250/5005]	Time  0.347 ( 0.333)	Data  0.002 ( 0.003)	Loss 3.5068e+00 (3.6184e+00)	Acc@1  28.12 ( 26.28)	Acc@5  52.34 ( 50.05)
Epoch: [1][3300/5005]	Time  0.323 ( 0.333)	Data  0.003 ( 0.003)	Loss 3.3031e+00 (3.6151e+00)	Acc@1  30.86 ( 26.33)	Acc@5  52.73 ( 50.11)
Epoch: [1][3350/5005]	Time  0.324 ( 0.333)	Data  0.002 ( 0.003)	Loss 3.4110e+00 (3.6123e+00)	Acc@1  34.38 ( 26.38)	Acc@5  53.91 ( 50.17)
Epoch: [1][3400/5005]	Time  0.322 ( 0.333)	Data  0.003 ( 0.003)	Loss 3.5301e+00 (3.6088e+00)	Acc@1  26.56 ( 26.44)	Acc@5  49.22 ( 50.24)
Epoch: [1][3450/5005]	Time  0.350 ( 0.333